In [1]:
import os 
import sys 
import builtins
from typing import List 

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

from functools import cache 
import numpy as np 
import pandas as pd 
import altair as alt 
from IPython.display import clear_output
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

from utils_notebook.utils import ddf, load_subgraph, remove_prefix
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    stack_order_expr, 
    wide_to_longwide,
    chart, 
)
from utils_notebook.queries import adjust_precision, QueryManager
from utils_notebook.testing import validate_season_series
from utils_notebook.css import css_tooltip_timeseries_multi_colored
from utils_notebook.queries import QueryManager
from utils_notebook.vega import condition_union, output_chart, possibly_override, XAXIS_DEFAULTS

In [2]:
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()

In [3]:
q = QueryManager(sg, bs) 

In [4]:
pool = bs.Query.metapoolOracles(first=100000, orderBy="season", orderDirection="asc")
df = sg.query_df(
    [
        pool.balanceA, 
        pool.balanceB, 
        pool.season, 
        pool.deltaB, 
        pool.timestamp, 
    ],
    pagination_strategy=ShallowStrategy
)
df = remove_prefix(df, "metapoolOracles_")
df.balanceA /= 10**6
df.balanceB /= 10**18
df.deltaB /= 10**6

In [5]:
# Reverse engineer pool reserves from quantites used in TWAP calculation. 
df['diff_a'] = (df.balanceA - df.balanceA.shift(1))
df['diff_b'] = (df.balanceB - df.balanceB.shift(1))
df['diff_timestamp'] = (df.timestamp - df.timestamp.shift(1))
df['reserves_3crv'] = df['diff_a'] / df.diff_timestamp
df['reserves_bean'] = df['diff_b'] / df.diff_timestamp
assert df.season.min() == 6076
df = df.iloc[1:,]
df.head()

,balanceA,balanceB,season,deltaB,timestamp,diff_a,diff_b,diff_timestamp,reserves_3crv,reserves_bean
1,1823002386736.906738,2030942247135.286865,6077,685226.464263,1659812464,46790827319.678223,50589125510.827393,3533.0,13243936.40523,14319027.883053
2,1869934826665.250244,2081743078278.517822,6078,694584.048577,1659816002,46932439928.343506,50800831143.230957,3538.0,13265245.881386,14358629.492151
3,1917706461458.546631,2133501113442.06958,6079,701977.691800,1659819600,47771634793.296387,51758035163.551758,3598.0,13277274.817481,14385223.780865
4,1965608261630.781006,2186128547794.177979,6080,803240.828322,1659823204,47901800172.234375,52627434352.108398,3604.0,13291287.506169,14602506.756967
5,2013881826521.260254,2239501123147.967285,6081,849932.384158,1659826833,48273564890.479248,53372575353.789307,3629.0,13302167.23353,14707240.38407


In [6]:
df_szns = q.query_seasons(extra_cols=['price'], where={"season_gte": 6074})[['season', 'price']]
df_szns = df_szns.rename(columns={"price": "price_bean"})
df_szns.head()

,season,price_bean
0,6074,1.022000
1,6075,1.070000
2,6076,1.050748
3,6077,1.051615
4,6078,1.051964


In [7]:
df = df.merge(df_szns, how="left", on="season")
df.head()

,balanceA,balanceB,season,deltaB,timestamp,diff_a,diff_b,diff_timestamp,reserves_3crv,reserves_bean,price_bean
0,1823002386736.906738,2030942247135.286865,6077,685226.464263,1659812464,46790827319.678223,50589125510.827393,3533.0,13243936.40523,14319027.883053,1.051615
1,1869934826665.250244,2081743078278.517822,6078,694584.048577,1659816002,46932439928.343506,50800831143.230957,3538.0,13265245.881386,14358629.492151,1.051964
2,1917706461458.546631,2133501113442.06958,6079,701977.691800,1659819600,47771634793.296387,51758035163.551758,3598.0,13277274.817481,14385223.780865,1.052462
3,1965608261630.781006,2186128547794.177979,6080,803240.828322,1659823204,47901800172.234375,52627434352.108398,3604.0,13291287.506169,14602506.756967,1.062113
4,2013881826521.260254,2239501123147.967285,6081,849932.384158,1659826833,48273564890.479248,53372575353.789307,3629.0,13302167.23353,14707240.38407,1.063713


In [8]:
# Approximation of pool TVL in $, since we can't compute this exactly without the price of 3Crv 
df['pool_tvl_usd'] = 2 * df.price_bean * (df.reserves_bean - df.deltaB)
df['bean_fraction'] = (df.price_bean * df.reserves_bean) / df.pool_tvl_usd
df['3crv_fraction'] = 1 - df.bean_fraction
df.tail()

,balanceA,balanceB,season,deltaB,timestamp,diff_a,diff_b,diff_timestamp,reserves_3crv,reserves_bean,price_bean,pool_tvl_usd,bean_fraction,3crv_fraction
2056,113329681736167.625,110771499979945.734375,8133,2090.227229,1667214011,50072062918.78125,49000911564.78125,3600.0,13908906.366328,13611364.32355,1.000150,27222630.974872,0.500077,0.499923
2057,113379766558663.0,110820488413638.9375,8134,-1450.907685,1667217611,50084822495.375,48988433693.203125,3600.0,13912450.69316,13607898.248112,0.999449,27203700.808824,0.499947,0.500053
2058,113429873739682.46875,110869454982437.921875,8135,-7660.251706,1667221211,50107181019.46875,48966568798.984375,3600.0,13918661.394297,13601824.666385,0.999449,27203972.183801,0.499719,0.500281
2059,113479980920701.953125,110918421551236.890625,8136,-7657.047186,1667224811,50107181019.484375,48966568798.96875,3600.0,13918661.394301,13601824.66638,0.999449,27203965.778284,0.499719,0.500281
2060,113530088101721.421875,110967388120035.875,8137,-7653.729622,1667228411,50107181019.46875,48966568798.984375,3600.0,13918661.394297,13601824.666385,0.999450,27203986.365777,0.499719,0.500281


In [9]:
df.timestamp = pd.to_datetime(df.timestamp, unit='s')
id_cols = ['timestamp']
value_cols = ['reserves_3crv', 'reserves_bean', 'deltaB', 'bean_fraction', '3crv_fraction', 'pool_tvl_usd']
df = df[id_cols + value_cols]
df = df.resample("D", on="timestamp").apply(lambda v: v.mean()).reset_index() 
df = wide_to_longwide(df, "timestamp", id_cols, value_cols)

In [10]:
selection_rule = alt.selection_single(
    name="sss", 
    fields=['timestamp'], nearest=True, on='mouseover', empty='none', clear='mouseout'
)

base = (
    alt.Chart(df)
    .encode(
        x=alt.X(
            "timestamp:O", 
            axis=alt.Axis(
                formatType="time", 
                ticks=False, 
                labelExpr="timeFormat(toDate(datum.value), '%b %e, %Y')", 
                labelOverlap=True, 
                labelSeparation=30, 
                labelPadding=5, 
                title='Date', 
                labelAngle=0, 
            )
        ), 
        color=alt.Color("variable:O", legend=None), 
        
    )
    .properties(width=500, height=250)
)

tooltip_formats = {
    "bean_fraction": ".1%", 
    "3crv_fraction": ".1%", 
    "pool_tvl_usd": "$,d"
}
rule = (
    # selection captures nearest timestamp (for current mouse position) 
    # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
    base
    .add_selection(selection_rule)
    .mark_rule(opacity=0)
    .encode(
        tooltip=(
            [alt.Tooltip(f'timestamp', timeUnit="yearmonthdate", title="date")] + 
            [alt.Tooltip(f'{m}', format=tooltip_formats.get(m, ",d")) for m in value_cols]
        ), 
    )
    
)
reserves = (
    base
    .transform_filter(condition_union("==", "|", ['reserves_3crv', 'reserves_bean']))
    .mark_line()
    .encode(
        y=alt.Y(
            "value:Q", 
            axis=alt.Axis(title="Reserves", format=".2~s"), 
            scale=alt.Scale(domainMin=12*1e6) # TODO: don't hardcode this
        )
    )
    .properties(title="Bean:3Crv Pool Reserves")
)
delta_b = (
    base
    .transform_filter("datum.variable == 'deltaB'")
    .mark_line()
    .encode(
        y=alt.Y(
            "value:Q", axis=alt.Axis(title="DeltaB", format=".2~s")
        )
    )
    .properties(title="Bean:3Crv Pool DeltaB")
)
balance = (
    base
    .transform_filter(condition_union("==", "|", ['bean_fraction', '3crv_fraction']))
    .mark_line()
    .encode(
        y=alt.Y(
            "value:Q", 
            axis=alt.Axis(title="Pool Balance", format="%"), 
            scale=alt.Scale(domainMin=.45, domainMax=.55), # TODO: don't hardcode this
        ), 
    )
    .properties(title="Bean:3Crv Pool Balance")
)
tvl = (
    base
    .transform_filter("datum.variable == 'pool_tvl_usd'")
    .mark_line()
    .encode(
        y=alt.Y(
            "value:Q", 
            axis=alt.Axis(title="Pool TVL", format="$.2~s"), 
        ), 
    )
    .properties(title="Bean:3Crv Pool TVL")
)
c = (
    (
        ((reserves + rule) & (delta_b + rule)) | 
        ((balance + rule) & (tvl + rule))
    )
    .resolve_scale(y="independent")
    .resolve_axis(y="independent")
)
c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [11]:
def chart(
    df: pd.DataFrame, 
    timestamp_col: str, 
    # metrics associated with left and right axes 
    lmetrics: List[str], 
    rmetrics: List[str] = None, 
    # strategy_fns to plot metrics on left and right axes 
    lstrategy = 'line', 
    rstrategy = 'line', 
    # mapping of strategy to stack layer order when rendering     
    lorder = None, 
    rorder = None, 
    # mapping of strategy to mark properties 
    lmark_kwargs = None, 
    rmark_kwargs = None, 
    # mapping of strategy to extra encodings 
    l_yscales = None, 
    r_yscales = None, 
    # Axis parameters 
    xaxis_kwargs = None, 
    xaxis_kwargs_override: bool = False, 
    yaxis_left_kwargs: dict = None, 
    yaxis_right_kwargs: dict = None, 
    # Chart parameters 
    title: str = '', 
    colors = None,      
    tooltip_formats = None, 
    tooltip_format_default: str = ",d", 
    tooltip_metrics = None, 
    dual_axes: bool = False, 
    show_exploit_rule: bool = True, 
    exploit_day: int = 17, # must be either 16 or 17
    width: int = 700, 
    hide_legend: bool = False, 
    selection_nearest: alt.selection = None, 
    create_selection: bool = True, 
    add_selection: bool = True, 
    return_selection: bool = False,     
    base_hook = None, 
): 
    """Creates a chart with a shared time axis and up to two y axes 
        
    Assumes that data is in long-wide format (i.e. df was processed with function wide_to_longwide)
    """
    rmetrics = rmetrics or []
    assert not set(lmetrics).intersection(set(rmetrics)), "Same metric on two axes"
    metrics = lmetrics + rmetrics
    tooltip_formats = tooltip_formats or {}
    xaxis_kwargs = possibly_override(
        xaxis_kwargs, XAXIS_DEFAULTS, override=xaxis_kwargs_override
    )
    yaxis_left_kwargs = yaxis_left_kwargs or {}
    yaxis_right_kwargs = yaxis_right_kwargs or {}
    lmark_kwargs = lmark_kwargs or {} 
    rmark_kwargs = rmark_kwargs or {}
    l_yscales = l_yscales or {}
    r_yscales = r_yscales or {}
    # Selection for nearest point. We either use an existing instance passed in by the user 
    # or create a new instance. Using an existing instance allows a selection to be shared 
    # across charts, which can be useful for creating interactions between linked views 
    assert not (create_selection and selection_nearest), "Can't create new selection while specifying existing one" 
    if create_selection: 
        selection_nearest = alt.selection_single(
            fields=[timestamp_col], nearest=True, on='mouseover', empty='none', clear='mouseout'
        )
    
    # Color scale is shared by metrics, regardless of what axis they belong to. User can 
    # either specify the desired colors for each metric or let defaults be used. 
    color_scale = (
        alt.Scale(domain=metrics, range=[colors[m] for m in metrics])
        if colors else 
        alt.Scale(domain=metrics)
    )
    
    
    base = (
        alt.Chart(df)
        .transform_calculate(stack_order=stack_order_expr("variable", metrics))
        .encode(x=alt.X(f"{timestamp_col}:O", axis=alt.Axis(**xaxis_kwargs)))
        .properties(title=title, width=width)
    )
    if base_hook: 
        base = base_hook(base)
        
    cbase = (
        base
        .encode(
            color=alt.Color("variable:N", scale=color_scale, legend=None if hide_legend else alt.Legend(title=None)), 
            order=alt.Order('stack_order:Q', sort='ascending'),
        )
    )

    class Strategies: 

        @staticmethod
        def _get_encode_kwargs(yscale_kwargs):
            yscale = alt.Scale(**yscale_kwargs) if yscale_kwargs else None
            encode_kwargs = dict() if not yscale else dict(scale=yscale)
            return encode_kwargs

        @staticmethod
        def line(base, axis, mark_kwargs, yscale_kwargs):
            encode_kwargs = Strategies._get_encode_kwargs(yscale_kwargs)
            return (
                base 
                .mark_line(**mark_kwargs)
                .encode(y=alt.Y("value:Q", axis=axis, **encode_kwargs))
            )
        
        @staticmethod
        def point(base, axis, mark_kwargs, yscale_kwargs):
            encode_kwargs = Strategies._get_encode_kwargs(yscale_kwargs)
            return (
                base 
                .mark_point(**mark_kwargs)
                .encode(y=alt.Y("value:Q", axis=axis, **encode_kwargs))
            )
        
        @staticmethod
        def stack_area(base, axis, mark_kwargs, yscale_kwargs):
            encode_kwargs = Strategies._get_encode_kwargs(yscale_kwargs)
            return (
                base 
                .mark_area(**mark_kwargs)
                .encode(y=alt.Y("value:Q", axis=axis, **encode_kwargs))
            )
            
        @staticmethod
        def stack_bar(base, axis, mark_kwargs, yscale_kwargs):
            encode_kwargs = Strategies._get_encode_kwargs(yscale_kwargs)
            return (
                base 
                .mark_bar(**mark_kwargs)
                .encode(y=alt.Y("value:Q", axis=axis, **encode_kwargs))
            )

    strategy_fns = {
        "line": Strategies.line, 
        "point": Strategies.point, 
        "stack_area": Strategies.stack_area, 
        "stack_bar": Strategies.stack_bar,
    }
    
    left_wrapper = dict(chart=None)
    right_wrapper = dict(chart=None)
    chart_specs = [
        (lstrategy, lmetrics, yaxis_left_kwargs, lorder, lmark_kwargs, l_yscales, left_wrapper), 
    ]
    if rmetrics: 
        chart_specs.append(
            (rstrategy, rmetrics, yaxis_right_kwargs, rorder, rmark_kwargs, r_yscales, right_wrapper)
        )
    for strategy, smetrics, axis_kwargs, order, mark_kwargs, yscales, chart_wrapper in chart_specs:
        match type(strategy): 
            case builtins.str: 
                # Apply a single strategy to all metrics on this axis 
                strat_fn = strategy_fns[strategy]
                strat_mark_kwargs = mark_kwargs.get(strategy, {})
                strat_yscales = yscales.get(strategy, {})
                chart_wrapper['chart'] = strat_fn(
                    cbase.transform_filter(condition_union("==", "|", smetrics)),
                    alt.Axis(**axis_kwargs), 
                    strat_mark_kwargs, 
                    strat_yscales, 
                ) 
            case builtins.list: 
                # Apply strategies on a per-metric basis 
                assert len(strategy) == len(smetrics)
                df_strategy_metric = pd.DataFrame(dict(strategy=strategy, metrics=smetrics))
                layers = []
                order_default = {
                    "stack_area": 0, 
                    "stack_bar": 1,
                    "line": 2, 
                    "point": 3, 
                }
                order = order or order_default
                ax = alt.Axis(**axis_kwargs)
                for strategy, df_sm in sorted(
                    df_strategy_metric.groupby("strategy"), key=lambda e: order[e[0]]
                ): 
                    sub_metrics = df_sm.metrics.values.tolist()
                    strat_fn = strategy_fns[strategy]
                    strat_mark_kwargs = mark_kwargs.get(strategy, {})
                    strat_yscales = yscales.get(strategy, {})
                    layer = strat_fn(
                        cbase.transform_filter(condition_union("==", "|", sub_metrics)), 
                        ax, 
                        strat_mark_kwargs,
                        strat_yscales
                    ) 
                    layers.append(layer)
                chart_wrapper['chart'] = alt.layer(*layers)  
            case _: 
                raise ValueError(f"Invalid strategy {strategy}")
            
    left = left_wrapper['chart']
    right = right_wrapper['chart']

    tooltip_metrics = tooltip_metrics or metrics 
    nearest = (
        # selection captures nearest timestamp (for current mouse position) 
        # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
        base
        .transform_pivot('variable', value='value', groupby=[timestamp_col])
        .mark_rule(color="#878787")
        .encode(
            tooltip=(
                [alt.Tooltip(f'{timestamp_col}:O', timeUnit="yearmonthdate", title="date")] + 
                [alt.Tooltip(f'{m}:Q', format=tooltip_formats.get(m, tooltip_format_default)) for m in tooltip_metrics]
            ), 
            opacity=alt.condition(selection_nearest, alt.value(1), alt.value(0))
        )
    )
    if add_selection: 
        nearest = nearest.add_selection(selection_nearest)
    

    assert exploit_day in [16, 17]
    rule_exploit = (
        # selection captures nearest timestamp (for current mouse position) 
        # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
        base
        .transform_pivot('variable', value='value', groupby=[timestamp_col])
        .transform_filter(f"""
            year(datum['{timestamp_col}']) === 2022 && 
            month(datum['{timestamp_col}']) === 3 && 
            date(datum['{timestamp_col}']) === {exploit_day} 
        """) # && warn(datetime(datum['{timestamp_col}']))
        .mark_rule(opacity=1, color='#474440', strokeDash=[2.5,1])
    )

    # Compose plot 
    if not rmetrics: 
        if show_exploit_rule: 
            c = left + rule_exploit + nearest
        else: 
            c = left + nearest
    else: 
        if show_exploit_rule: 
            # It matters that the rules are layered with right instead of left, not sure why. 
            # Parentheses are important in case where dual_axes is True 
            c = left + (right + rule_exploit + nearest)
        else: 
            # It matters that the rules are layered with right instead of left, not sure why. 
            # Parentheses are important in case where dual_axes is True 
            c = left + (right + nearest)
    if dual_axes: 
        assert rmetrics, "Can't have two axes if you didn't specify rmetrics" 
        c = (
            c
            .resolve_scale(y="independent")
            .resolve_axis(y="independent")
        )
    return c if not return_selection else (c, selection_nearest)

In [12]:
df.head()

,timestamp,variable,value,reserves_3crv,reserves_bean,deltaB,bean_fraction,3crv_fraction,pool_tvl_usd
0,2022-08-06,reserves_3crv,1.327598e+07,1.327598e+07,1.447453e+07,746992.283424,0.527194,0.472806,2.900371e+07
1,2022-08-06,reserves_bean,1.447453e+07,1.327598e+07,1.447453e+07,746992.283424,0.527194,0.472806,2.900371e+07
2,2022-08-06,deltaB,7.469923e+05,1.327598e+07,1.447453e+07,746992.283424,0.527194,0.472806,2.900371e+07
3,2022-08-06,bean_fraction,5.271938e-01,1.327598e+07,1.447453e+07,746992.283424,0.527194,0.472806,2.900371e+07
4,2022-08-06,3crv_fraction,4.728062e-01,1.327598e+07,1.447453e+07,746992.283424,0.527194,0.472806,2.900371e+07


In [18]:
width = 450 
tooltip_metrics = [
    'reserves_3crv', 'reserves_bean',
    'bean_fraction', '3crv_fraction',
    'deltaB', 
    'pool_tvl_usd', 
]
tooltip_formats = {
    "bean_fraction": ".1%", 
    "3crv_fraction": ".1%", 
    "pool_tvl_usd": "$,d"
}


chart_reserves, selection_nearest = chart(
    df, 
    "timestamp", 
    lmetrics=['reserves_3crv', 'reserves_bean'], 
    lstrategy='line', 
    l_yscales={"line": dict(domainMin=12*1e6)}, # TODO: don't hardcode this
    title="Bean:3Crv Pool Reserves", 
    yaxis_left_kwargs=dict(title="Reserves", format=".2~s"),    
    create_selection=True, 
    add_selection=True, 
    return_selection=True, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    width=width, 
)
chart_balance = chart(
    df, 
    "timestamp", 
    lmetrics=['bean_fraction', '3crv_fraction'], 
    lstrategy='line', 
    title="Bean:3Crv Pool Balance", 
    yaxis_left_kwargs=dict(title="Balance Ratio", format="%"),    
    l_yscales={"line": dict(domainMin=.45, domainMax=.55)}, # TODO: don't hardcode this
    selection_nearest=selection_nearest, 
    create_selection=False, 
    add_selection=True, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    width=width, 
)
chart_deltab = chart(
    df, 
    "timestamp", 
    lmetrics=['deltaB'], 
    lstrategy='line', 
    title="Bean:3Crv Pool DeltaB", 
    yaxis_left_kwargs=dict(title="DeltaB", format=".2~s"),    
    # l_yscales={"line": dict(domainMin=.45, domainMax=.55)}, # TODO: don't hardcode this
    selection_nearest=selection_nearest, 
    create_selection=False, 
    add_selection=True, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    width=width, 
)
chart_tvl = chart(
    df, 
    "timestamp", 
    lmetrics=['pool_tvl_usd'], 
    lstrategy='line', 
    title="Bean:3Crv Pool TVL", 
    yaxis_left_kwargs=dict(title="Pool TVL", format="$.2~s"),    
    # l_yscales={"line": dict(domainMin=.45, domainMax=.55)}, # TODO: don't hardcode this
    selection_nearest=selection_nearest, 
    create_selection=False, 
    add_selection=True, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    width=width, 
)

# base = (
#     alt.Chart(df)
#     .encode(
#         x=alt.X(
#             "timestamp:O", 
#             axis=alt.Axis(
#                 formatType="time", 
#                 ticks=False, 
#                 labelExpr="timeFormat(toDate(datum.value), '%b %e, %Y')", 
#                 labelOverlap=True, 
#                 labelSeparation=30, 
#                 labelPadding=5, 
#                 title='Date', 
#                 labelAngle=0, 
#             )
#         ), 
#         color=alt.Color("variable:O", legend=None), 
        
#     )
#     .properties(width=500, height=250)
# )

# tooltip_formats = {

# }
# rule = (
#     # selection captures nearest timestamp (for current mouse position) 
#     # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
#     base
#     .add_selection(selection_rule)
#     .mark_rule(opacity=0)
#     .encode(
#         tooltip=(
#             [alt.Tooltip(f'timestamp', timeUnit="yearmonthdate", title="date")] + 
#             [alt.Tooltip(f'{m}', format=tooltip_formats.get(m, ",d")) for m in value_cols]
#         ), 
#     )
    
# )




# c = (
#     (
#         ((reserves + rule) & (delta_b + rule)) | 
#         ((balance + rule) & (tvl + rule))
#     )
#     .resolve_scale(y="independent")
#     .resolve_axis(y="independent")
# )
# c

c = (
    alt.vconcat(
        (chart_reserves | chart_balance), 
        (chart_deltab | chart_tvl),
    )
    .resolve_scale(y="independent")
    .resolve_axis(y="independent")
)
c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [ ]:
# TODO: Update tooltip css once we get that fixed on the frontend. 
output_chart(c)